In [ ]:
import os
import time
filename = 'DOT_Traffic_Speeds_NBE.csv'

Pandas

In [ ]:
import pandas as pd

In [ ]:
start = time.time()
pandasDF = pd.read_csv(filename)
finish = time.time()
print(f'Time to Completion: {(finish-start)//60} minutes and {((finish-start)%60):2} seconds.')

Dask

In [ ]:
import dask.dataframe as dd

In [ ]:
start = time.time()
daskDF = dd.read_csv(filename)
finish = time.time()
print(f'Time to Completion: {(finish-start)//60} minutes and {((finish-start)%60):2} seconds.')

Modin/Ray

In [ ]:
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as mpd

In [ ]:
start = time.time()
modinDF = mpd.read_csv(filename)
finish = time.time()
print(f'Time to Completion: {(finish-start)//60} minutes and {((finish-start)%60):2} seconds.')

Data Validation

In [ ]:
daskDF = dd.read_csv(filename)

In [ ]:
daskDF.head()

In [ ]:
cols = daskDF.columns.values
print(f'Given Columns: {cols}\n')
for i in range(len(daskDF.columns)):
    specialRemoved = ''.join([a for a in daskDF.columns[i] if (a.isalnum())])
    cols[i] = specialRemoved
dictCols = {i:cols[i] for i in range(len(cols))}
daskDF.rename(columns=dictCols)
print(f'Parsed Columns: {daskDF.columns.values.tolist()}')

YAML

In [ ]:
import yaml

In [ ]:
%%writefile data.yaml
fileType: csv
fileName: DOT_Traffic_Speeds_NBE
readDelimiter: ","
writeDelimiter: "|"
columns: ['ID', 'SPEED', 'TRAVELTIME', 'STATUS', 'DATAASOF', 'LINKID', 'LINKPOINTS', 'ENCODEDPOLYLINE', 'ENCODEDPOLYLINELVLS', 'OWNER', 'TRANSCOMID', 'BOROUGH', 'LINKNAME']

In [ ]:
data = open('data.yaml', 'r')
yml = yaml.load(data, yaml.Loader)
print(yml)
print('Number of Columns:', len(yml['columns']))

File Writing & Summary

In [ ]:
daskDF.to_csv(f"{yml['fileName']}.gz", compression='gzip', sep=yml['writeDelimiter'])

print(f"Number of Columns: {len(daskDF.columns.values)}")
print(f"Number of Rows: {len(daskDF)}")
print('File Size:', os.path.getsize(f"{yml['fileName']}.gz"))